In [45]:
# add path to module so can import mlfinder code
import sys
sys.path = ['/Users/judahluberto/mlfinder'] + sys.path

# ignore future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# basic imports
from astropy.io import ascii
from astropy.io import fits
from astropy.table import Table
from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u

import math

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

import statistics

from PyAstronomy import pyasl

import os

# datalab imports for background stars
import dl
from dl import queryClient as qc
from dl.helpers.utils import convert

# import mlfinder module
from mlfinder.bd import BrownDwarf
from mlfinder.fields import Fields
from mlfinder.events import FindEvents

In [114]:
# get dataframe

df = Table.read('J_AcA_68_351_events.dat.fits', format='fits')
df = df.to_pandas()

df['Disc'] = df['Disc'].str.decode("utf-8")

# make a copy of df, so can reference later
df_all = df.copy()

In [115]:
# rename and select columns in dataframe

rename_dict = {'Disc':'object_name', 
               'RALdeg':'ra', 
               'DELdeg':'dec',
               'EpL': 'EpL',
               't0': 'Prev_Pred_Min_Sep',
               'PlxL':'pi', 
               'pmRA*L':'mu_alpha', 
               'pmDEL':'mu_delta', 
               'e_RALdeg':'pm_ra',
               'e_DELdeg':'pm_dec',
               'e_pmRA*L':'pm_mu_alpha',
               'e_pmDEL':'pm_mu_delta',
               'e_PlxL': 'pm_pi'}

df = df.rename(columns = rename_dict)

# now keep those columns too -- for simplicity!
keep_list = ['object_name', 'ra', 'dec', 'EpL', 'pi', 'mu_alpha', 'mu_delta', 'pm_mu_alpha', 'pm_mu_delta', 'pm_pi', 'Prev_Pred_Min_Sep']

df = df[keep_list]

# change name of EpL (already had a t0, so have to convoluded way it)
df = df.rename(columns = {'EpL': 't0'})

# add ra and dec error
df['pm_ra'] = 0
df['pm_dec'] = 0

In [125]:
df

,object_name,ra,dec,t0,pi,mu_alpha,mu_delta,pm_mu_alpha,pm_mu_delta,pm_pi,Prev_Pred_Min_Sep,pm_ra,pm_dec
0,PSO_J076.7092+52.6087,76.709193,52.608725,2010.87847,59.28,57.0,-207.2,4.6,3.0,5.27,2044.71923,0,0
1,WISEA_J053257.29+041842.5,83.238910,4.311572,2012.36983,31.63,276.2,-440.6,1.7,2.0,3.81,2052.51581,0,0
2,2MASS_J05591914-1404488,89.831895,-14.081459,2011.90516,96.60,569.0,-339.6,2.8,2.0,1.00,2056.08378,0,0
3,WISE_J070159.79+632129.2,105.499205,63.358112,2010.47422,60.53,-17.3,-261.7,3.1,4.4,6.19,2062.25955,0,0
4,DENIS-P_J0751164-253043,117.815211,-25.511497,2011.44597,56.30,-880.7,150.1,1.2,1.5,0.09,2045.78042,0,0
5,VVV_BD001,261.667255,-27.634409,2012.02303,57.00,-548.7,-329.6,2.2,4.4,4.00,2036.93087,0,0
6,2MASS_J18284076+1229207,277.169077,12.488828,2011.31003,16.36,-244.4,-90.2,2.3,1.4,2.72,2061.19042,0,0
7,vB_10,289.238239,5.146342,2010.72041,168.15,-594.9,-1364.1,2.2,1.9,0.50,2027.06995,0,0
8,vB_10,289.238239,5.146342,2010.72041,168.15,-594.9,-1364.1,2.2,1.9,0.50,2065.73963,0,0
9,vB_10,289.238239,5.146342,2010.72041,168.15,-594.9,-1364.1,2.2,1.9,0.50,2055.34675,0,0


In [94]:
month_dict = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

def time_to_good(date, init_format='mjd'):
    # create Time instantiation to turn from MJD to iso
    t = Time(float(date), format=init_format)
    t.format = 'iso'
    t = t.value
    
    # split into month, day, year
    t_split = t.split()[0]
    t_split = t_split.split('-')

    # final reformat
    t = t_split[0] + '-' + str(month_dict[int(t_split[1])]) + '-' + t_split[2]
    
    return t

In [119]:
# initliaze a dataframe to append events to, and mc dict of mass uncertainties if done
all_events = pd.DataFrame()
stars_count = 0
for index, row in df.iterrows():
    # print to keep us updated
    print('Working on dwarf {}, index {}...'.format(row.object_name, index))

    # turn row into a pandas dataframe for ease
    row = row.to_frame().T

    # get time in good format
    t = time_to_good(row.t0, init_format='decimalyear')
    
    t_start = time_to_good(row.Prev_Pred_Min_Sep - 5, init_format='decimalyear')
    t_end = time_to_good(row.Prev_Pred_Min_Sep + 5, init_format='decimalyear')
    
    # create brown dwarf class and find a future path
    bd = BrownDwarf(row, observ_date=t)
    bd_path = bd.find_path(start=t_start, end=t_end, step='7days')

    # find the background stars from the Legacy Survey
    stars = Fields(bd=bd)
    
    if len(stars.stars) > 0:
        stars_count += 1
    
    # find events with mass uncertainty lower than 1000 Mjup
    events = FindEvents(bd, stars, 1000)

    # append to dataframe
    all_events = all_events.append(events.event_table)

Working on dwarf PSO_J076.7092+52.6087    , index 0...


/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


Working on dwarf WISEA_J053257.29+041842.5, index 1...


/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


Working on dwarf 2MASS_J05591914-1404488  , index 2...


...

/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


Working on dwarf vB_10                    , index 25...


/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


Working on dwarf GJ_1245B                 , index 26...


/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [124]:
# reformat all_events
event_objects, event_sep, event_deltam = list(), list(), list()
for index, row in all_events.iterrows():
    # grab item 
    event_objects.append(row.object_name.item())
    
    # sep and delta_m
    if type(row.sep) is np.ndarray:
        event_sep.append(row.sep[0])
        event_deltam.append(row.delta_m[0])
    else:
        event_sep.append(row.sep)
        event_deltam.append(row.delta_m) 
        
all_events['object_name'] = event_objects
all_events['sep'] = event_sep
all_events['delta_m'] = event_deltam

In [144]:
all_events

,object_name,sep,delta_m,bd_ra,bd_dec,ls_id,bs_ra,bs_dec,mag,time_of_min,gaia_id
0,WISE_J070159.79+632129.2,0.180893,76.865448,105.498628,63.354414,8797233579233248,105.498521,63.354398,21.306847,2061.340178,NaN
0,2MASS_J05441150-2433018,0.193023,102.808931,86.049201,-24.554746,8796105718433859,86.049248,-24.554715,19.174593,2021.422313,NaN
